In [1]:
# IMPORT LIBRARIES #
import pandas as pd
import pygsheets
import numpy as np

In [2]:
# FLAGS #

#Set to 1 to print diagnostic info
debugFlag = 0
#Set to 1 to pull info from RawNames sheet, join GPS, update, and export
updateGPSFlag = 0

In [3]:
# IMPORT DATA #

gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')

if debugFlag:
    for x in sh:
        print(x)

for x in sh:
    title = x.title
    if(title=="DeliveriesOld"):
        import_sheets_OldDelDF = x.get_as_df()
    elif(title=="Deliveries"):
        import_sheets_DelDF = x.get_as_df()
    elif(title=="Days"):
        import_sheets_DaysDF = x.get_as_df()
    elif(title=="Weeks"):
        import_sheets_WeeksDF = x.get_as_df()
    elif(title=="RawNames"):
        import_sheets_RawNamesDF = x.get_as_df()
    elif(title=="GPS"):
        #for import
        import_sheets_GPSDF = x.get_as_df()
        #for export
        if updateGPSFlag:
            import_wks_gps = x
    else:
        print("Worksheet not stored: ", x)

Worksheet not stored:  <Worksheet 'Import' index:6>
Worksheet not stored:  <Worksheet 'Lifetime Deliveries' index:7>
Worksheet not stored:  <Worksheet 'Days Old' index:8>
Worksheet not stored:  <Worksheet 'Weeks Old' index:9>


In [4]:
# Allows for disjoint cells, one for import, one for reset to import value (for ease of testing)

#DF from sheets
sheets_OldDelDF   = import_sheets_OldDelDF.copy()
sheets_DelDF      = import_sheets_DelDF.copy()
sheets_DaysDF     = import_sheets_DaysDF.copy()
sheets_WeeksDF    = import_sheets_WeeksDF.copy()
sheets_RawNamesDF = import_sheets_RawNamesDF.copy()
sheets_GPSDF      = import_sheets_GPSDF.copy()

In [5]:
# FUNCTIONS #

# TODO: develop function to take avg distance and duration for stacked orders
def processStacks(df):
    return df

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)
        
def hours_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)

def getStartHour(df):
    df["Start Hour"] = -1
    for index, row in df.iterrows():
        startHour = row["Start Time"]
        startHour = int(startHour[:startHour.index(":")])
        df.at[index, "Start Hour"] = startHour

def addStartAndEndDate(df):
    lastRow = df.iloc[[-1]]
    df = df[:-1]
    df["Start Date"] = -1
    df["End Date"]   = -1
    for index, row in df.iterrows():
        tempVar = row["Dates"]
        tempVar1 = tempVar[:tempVar.index(" ")]
        tempVar2 = tempVar[tempVar.index("- ")+2:]
        df.at[index, "Start Date"] = tempVar1
        df.at[index, "End Date"] = tempVar2
    df = pd.concat([df, lastRow], ignore_index=True)
    df.reset_index(drop=True, inplace=True)
    return df

#Currently only valid for 2023
def date_to_datetime(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
    df[column] = pd.to_datetime(df[column])
        
        
#Currently only valid for 2023
# Currently unused function
# Would we actually want this to have different dates? it's really the date the shift started
# So maybe, keep date same, but certain analysis might require that difference to be handled
def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [6]:
# UPDATE GPS WORKSHEET #

if updateGPSFlag:
    testingDF = sheets_RawNamesDF.copy()
    testingDF = testingDF.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
    testingDF = testingDF[["Restaurant Name", "RAW DATA", "Note"]]
    newNamesDF = testingDF.merge(sheets_GPSDF, how='left')
    for index, row in newNamesDF.iterrows():
        if isinstance(row['Address'], float):
            if not row['Note'] == "":
                newNamesDF.at[index, "Address"] = row["Note"]
            else:
                print("Note not found for:", index)
        if (row['Address'] == ""):
            newNamesDF.at[index, "Resolved"] = "0"
        else:
            newNamesDF.at[index, "Resolved"] = "1"
    newNamesDF = newNamesDF.drop(columns=["Note"])
    import_wks_gps.set_dataframe(newNamesDF,(1,1))

In [7]:
# START PROCESSING #

In [8]:
# FUNCTIONS AND OTHER SHORT PROCESSING STEPS #


#V1/V2
dollars_to_floats(sheets_OldDelDF, ["Total", "Base", "Tip", "Peak Bonus"])
date_to_datetime(sheets_OldDelDF, "Date")
sheets_OldDelDF["Day of Week"] = sheets_OldDelDF["Date"].dt.day_name()
#V3 only
dollars_to_floats(sheets_DelDF, ["Total", "Base", "Tip", "Peak Bonus"])
getStartHour(sheets_DelDF)
date_to_datetime(sheets_DelDF, "Date")
sheets_DelDF["Day of Week"] = sheets_DelDF["Date"].dt.day_name()
#Days
dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])
hours_to_floats(sheets_DaysDF, ["Dash", "Active"])
date_to_datetime(sheets_DaysDF, "Date")
sheets_DaysDF["Day of Week"] = sheets_DaysDF["Date"].dt.day_name()
sheets_DaysDF["HourlyRate"] = sheets_DaysDF["Total"]/sheets_DaysDF["Dash"]
#Weeks
dollars_to_floats(sheets_WeeksDF, ["Total", "Base", "Tip", "Adj pay", "Other", "Pre Total"])
hours_to_floats(sheets_WeeksDF, ["Dash", "Active"])
sheets_WeeksDF = addStartAndEndDate(sheets_WeeksDF)
# pretotalactiveratio refers to weekly pre adjustment pay / time spent on deliveries
sheets_WeeksDF["pretotalactiveratio"] = round(sheets_WeeksDF["Pre Total"]/sheets_WeeksDF["Active"], 2)
sheets_WeeksDF["HourlyRate"] = sheets_WeeksDF["Total"]/sheets_WeeksDF["Dash"]

/tmp/ipykernel_5154/3873599355.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_5154/3873599355.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_5154/3873599355.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_5154/3873599355.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular exp

In [9]:
# BEGIN ADJ PAY SECTION (v3 only) #

# This establishes the estimated pay adjustment estimates by delivery, date, and week
# This is calculated using time spent delivering and distance approximations
# The real adjusted pay is based on weekly values
# I suspect the primary cause for difference between estimate and real values stems from 
#how I estimate mileage vs how the app tracks mileage

In [10]:
#Per delivery estimations


sheets_DelDF["Duration"] = pd.to_numeric(sheets_DelDF["Duration"])
sheets_DelDF["Est Adj"] = (sheets_DelDF["Duration"]/60*18
                           + sheets_DelDF["Distance"]*(0.34) ) - sheets_DelDF["Base"]
#Duration is over 60 because its value is in minutes, values from day and week are in hours

In [11]:
#Per day estimations
sheets_DaysDF1 = sheets_DaysDF.copy()
sheets_DaysDF1 = sheets_DaysDF1[sheets_DaysDF1["Version"] == 3]
sheets_DaysDF1["Est Adj"] = (sheets_DaysDF1["Active"]*(18) 
                            + sheets_DaysDF1["Active Mileage"]*(0.34) ) - sheets_DaysDF1["Base"]

In [12]:
#Per week estimations
sheets_WeeksDF1 = sheets_WeeksDF.copy()
sheets_WeeksDF1 = sheets_WeeksDF1[sheets_WeeksDF1["Version"] == 3] #totals doesn't have v3 so its dropped
sheets_WeeksDF1["Est Adj"] = (sheets_WeeksDF1["Active"]*(18) 
                            + sheets_WeeksDF1["Active Mileage"]*(0.34) ) - sheets_WeeksDF1["Base"]

In [13]:
# END ADJ PAY SECTION #

In [14]:
# REBUILD V1,2 + V3 SETS

# Deliveries #
sheets_result = pd.concat([sheets_OldDelDF, sheets_DelDF], sort=False)
sheets_result.reset_index(drop=True, inplace=True)

# V1, V2 #
sheets_DaysDF0 = sheets_DaysDF[sheets_DaysDF["Version"] != 3]
sheets_WeeksDF0 = sheets_WeeksDF[sheets_WeeksDF["Version"] != 3]
total_S = sheets_WeeksDF0.iloc[[-1]]
sheets_WeeksDF0 = sheets_WeeksDF0[:-1]

# Days #
sheets_DaysDF = pd.concat([sheets_DaysDF0, sheets_DaysDF1], sort=False)
sheets_DaysDF.reset_index(drop=True, inplace=True)

# Weeks #
sheets_WeeksDF = pd.concat([sheets_WeeksDF0, sheets_WeeksDF1], sort=False)
sheets_WeeksDF = pd.concat([sheets_WeeksDF, total_S], ignore_index=True)
sheets_WeeksDF.reset_index(drop=True, inplace=True)

In [15]:
if debugFlag:
    print(list(sheets_result.columns.values))
    print(sheets_result.dtypes)
    print()
    print(list(sheets_DaysDF.columns.values))
    print(sheets_DaysDF.dtypes)
    print()
    print(list(sheets_WeeksDF.columns.values))
    print(sheets_WeeksDF.dtypes)

In [16]:
# END PROCESSING #
# START INITIAL EXPLORATION #

In [17]:
#Percentage of deliveries taken that tip
numUntipped = (sheets_result["Tip"] == 0.0).sum()
numTipped = (sheets_result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 4)
print(percTipped*100, "% of accepted deliveries included tip")

#Average tip, including and excluding untipped deliveries
TipAvg = round(sheets_result["Tip"].mean(), 2)
print("$", TipAvg, "average tip value including orders with no tip")
TipAvg2 = round(sheets_result[sheets_result["Tip"] !=0]["Tip"].mean(), 2)
print("$", TipAvg2, "average tip value excluding orders with no tip")

# Overall hourly rate
hourlyRate = round(sheets_WeeksDF.iloc[-1]["Total"]/sheets_WeeksDF.iloc[-1]["Dash"], 2)
print("Overall hourly pay rate is $", hourlyRate)

# Max and Average values by Delivery, Day, and Week
# Some of these values are wrong, because I have not yet implemented a stack value handler
print()
print("Daily Max Values")
print(sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].max())
print(sheets_DaysDF[sheets_DaysDF["Version"] == 3][["Total Mileage", "Active Mileage"]].max(), "\n" )
print("Daily Average Values")
print(sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].mean())
print(sheets_DaysDF[sheets_DaysDF["Version"] == 3][["Total Mileage", "Active Mileage"]].mean(), "\n" )
print()
# Some of these values are wrong, because I have not yet implemented a stack value handler
print("Weekly Max Values")
#
print(sheets_WeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].max())
#
print(sheets_WeeksDF[sheets_WeeksDF["Version"] == 3][["Total Mileage", "Active Mileage"]].max(), "\n")
print("Weekly Average Values")
#
print(sheets_WeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].mean())
#
print(sheets_WeeksDF[sheets_WeeksDF["Version"] == 3][["Total Mileage", "Active Mileage"]].mean(), "\n")
print()
# Some of these values are wrong, because I have not yet implemented a stack value handler
print("Delivery Max Values")
#
print(sheets_DelDF[["Distance", "Duration"]].max())
print(sheets_result[["Total"]].max(), "\n")
print("Delivery Average Values")
#
print(sheets_DelDF[["Distance", "Duration"]].mean())
print(sheets_result[["Total"]].mean(), "\n")

# END INITIAL EXPLORATION #

90.68 % of accepted deliveries included tip
$ 4.12 average tip value including orders with no tip
$ 4.54 average tip value excluding orders with no tip
Overall hourly pay rate is $ 25.54

Daily Max Values
Total         333.57
Base          191.50
Tip           149.53
Active          9.67
Dash           13.60
Deliveries     33.00
dtype: float64
Total Mileage       151
Active Mileage    229.2
dtype: object 

Daily Average Values
Total         162.688254
Base           91.123016
Tip            71.565238
Active          5.154444
Dash            7.406667
Deliveries     17.365079
dtype: float64
Total Mileage     95.300000
Active Mileage    91.393333
dtype: float64 


Weekly Max Values
Base           5740.75
Tip            4508.61
Adj pay        1652.51
Pre Total     10249.36
Total         11934.62
Active          325.45
Dash            467.23
Deliveries     1094.00
dtype: float64
Total Mileage       633
Active Mileage    619.4
dtype: object 

Weekly Average Values
Base           604.289474
T

In [18]:
# Restaurants DF #

tempDF = sheets_result.copy()
#The line below drops restaurants that only appear once, we may no longer want this to happen
#Commented out for now, may make sense to use when considering outliers for averages
#tempDF = tempDF[sheets_result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Base": [np.mean, np.sum], 
                                                  "Tip": [np.mean, np.sum],
                                                  "Total": [np.mean, np.sum],
                                                 'Restaurant Name': 'size'})
tempDF.columns = ['Avg Base', 'Sum Base', 'Avg Tip', 
                  'Sum Tip', 'Avg Total', 'Sum Total', "Deliveries"]

tempDF = tempDF.round({'Avg Base': 2, 
                       'Sum Base': 2,
                       'Avg Tip': 2,
                       'Sum Tip': 2,
                       'Avg Total': 2,
                       'Sum Total': 2})
tempDF1 = tempDF.index.copy() # All restaurant names visited
restaurantsDF = tempDF.copy()
# Change sort value here
restaurantsDF.sort_values("Sum Total", ascending=False)

,Avg Base,Sum Base,Avg Tip,Sum Tip,Avg Total,Sum Total,Deliveries
Restaurant Name,,,,,,,
Chick-fil-A,5.74,717.75,3.61,450.89,9.35,1168.64,125
Taco Bell,4.79,315.87,3.44,226.75,8.70,574.12,66
McDonald's,4.60,326.50,3.29,233.92,7.91,561.42,71
Dave's Hot Chicken,5.26,263.25,4.29,214.50,9.56,477.75,50
Chipotle,4.69,248.50,3.52,186.42,8.24,436.92,53
...,...,...,...,...,...,...,...
Everbowl,4.00,4.00,2.00,2.00,6.00,6.00,1
Kung Fu Noodle,5.50,5.50,0.00,0.00,5.50,5.50,1
North Park Produce Bakery and Grill,3.00,3.00,2.00,2.00,5.00,5.00,1


In [19]:
# Restaurants DF #

#Dollars Earned vs Miles Driven is another factor I wanted to consider in the overall earnings equation

tempDF = sheets_result[sheets_result["Version"] == 3].copy()
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Distance": "sum"})
tempDF = tempDF.round({'Base': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Distance"]
tempDF2 = tempDF.index.copy() #All restaurants from v1 and v2
restaurantsDF = restaurantsDF.join(tempDF["DollarMilesRatio"])

In [20]:
restaurantsDF

,Avg Base,Sum Base,Avg Tip,Sum Tip,Avg Total,Sum Total,Deliveries,DollarMilesRatio
Restaurant Name,,,,,,,,
7-Eleven,3.96,146.37,2.32,86.00,6.42,237.37,37,1.256742
Acai Superfruits,5.25,5.25,5.00,5.00,10.25,10.25,1,NaN
Advanced Auto Parts,6.95,34.75,0.00,0.00,6.95,34.75,5,2.151639
Albertsons,4.58,13.75,12.39,37.17,17.64,52.92,3,3.882955
Angry Chickz,6.25,6.25,5.00,5.00,11.25,11.25,1,1.424051
...,...,...,...,...,...,...,...,...
Wild Burger,5.00,5.00,4.00,4.00,9.00,9.00,1,NaN
Wings-N-Things,4.56,18.25,3.62,14.50,8.19,32.75,4,1.867089
Wingstop,5.33,16.00,3.83,11.50,9.17,27.50,3,1.536313


In [21]:
# Select the restaurants from the complete list that do not 'yet' appear in v3
tempDF3 = tempDF1[~tempDF1.isin(tempDF2)].dropna(how = 'all')
tempDF3

Index(['Acai Superfruits', 'Beach Hut Deli', 'Bloom Haus',
       'Brooklyn Boyz Pizza', 'Chili's Grill & Bar', 'Chin's Szechwan',
       'Cream Of The Crop', 'Epic Wings', 'Erika's Mexican Food & Seafood',
       'Everbowl', 'FTD', 'Firehouse Subs', 'Greek Corner Cafe',
       'Guanatos City Mexican Restaurant', 'Handel's Homemade Ice Cream',
       'Harney Sushi', 'Jamba', 'Joe's Crab Shack', 'Krispy Kreme',
       'Mikko Sushi', 'North Park Produce Bakery and Grill',
       'Ocean Thai Cuisine', 'PETCO', 'Pit Stop Diner', 'Pollos Maria',
       'Quik Wok', 'Sizzler', 'Sonic Drive-In', 'Super Mega Dilla',
       'TZONE Oceanside', 'Tapioca Express', 'TenderFix by Noah Schnapp',
       'The Cheesecake Factory', 'The Cheesesteak Grill', 'The Meltdown',
       'The Poke Cafe', 'Tony's Fresh Mexican Food',
       'Umi Japanese Grill & Sushi', 'Village Kabob', 'Wienerschnitzel',
       'Wild Burger'],
      dtype='object', name='Restaurant Name')

In [22]:
# Day of Week DF #

# NOTE: This excludes pay adjustments

tempDF = sheets_DaysDF.copy()
tempDF = tempDF.groupby(["Day of Week"]).agg(
    {"Dash": "sum", 
     "Total": [np.mean, np.sum], 
     "Tip": "sum", 
     "Deliveries": "sum"})
tempDF.columns = ['Sum Dash', 'Avg Daily Total', 'Sum Total', 'Sum Tip', 'Sum Deliveries']
tempDF["$/hr"] = tempDF["Sum Total"]/tempDF["Sum Dash"]
tempDF["Avg Delivery Tip"] = tempDF["Sum Tip"]/tempDF["Sum Deliveries"]
tempDF["Avg Delivery Total"] = tempDF["Sum Total"]/tempDF["Sum Deliveries"]
tempDF = tempDF.round({'Sum Dash': 2, 
                       'Avg Total': 2, 
                       'Sum Total':2, 
                       'Sum Tip': 2,
                       '$/hr': 2,
                       'AvgTip': 2,
                       'AvgTotal':2})
display(tempDF)

,Sum Dash,Avg Daily Total,Sum Total,Sum Tip,Sum Deliveries,$/hr,Avg Delivery Tip,Avg Delivery Total
Day of Week,,,,,,,,
Friday,142.37,188.516471,3204.78,1460.28,333,22.51,4.385225,9.623964
Monday,17.85,125.823333,377.47,158.97,41,21.15,3.877317,9.206585
Saturday,116.18,178.880000,2683.20,1179.95,279,23.10,4.229211,9.617204
Sunday,49.58,147.347500,1178.78,545.53,122,23.78,4.471557,9.662131
Thursday,72.83,140.773636,1548.51,656.01,176,21.26,3.727330,8.798352
Wednesday,67.81,139.624444,1256.62,507.87,143,18.53,3.551538,8.787552


In [23]:
# UNDER CONSTRUCTION #

In [24]:
sheets_WeeksDF.tail()

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Total Mileage,Active Mileage,Version,WeekID,Start Date,End Date,pretotalactiveratio,HourlyRate,Est Adj
14,7/17 - 7/23,454.75,425.11,151.31,3.38,879.86,1034.55,26.22,37.30,91,471,438.6,3,16,7/17,7/23,33.56,27.735925,166.334
15,7/24 - 7/30,278.25,209.79,71.96,0.00,488.04,560.00,14.82,19.52,50,285,271.8,3,17,7/24,7/30,32.93,28.688525,80.922
16,7/31 - 8/6,248.75,183.44,74.75,0.00,432.19,506.94,13.75,18.57,44,272,386.2,3,18,7/31,8/6,31.43,27.298869,130.058
17,8/7 - 8/13,149.50,109.50,0.00,0.00,259.00,259.00,7.62,10.15,26,141,120.6,3,19,8/7,8/13,33.99,25.517241,28.664
18,TOTALS:,5740.75,4508.61,1652.51,32.75,10249.36,11934.62,325.45,467.23,1094,2859,2741.76,-,,NaN,NaN,31.49,25.543351,NaN


In [25]:
sheets_DaysDF.tail()

,Date,Total,Base,Tip,Start (24),End (24),Active,Dash,Offers,Deliveries,Mileage Start,Mileage End,Total Mileage,Active Mileage,Version,Pauses,WeekID,Day of Week,HourlyRate,Est Adj
58,2023-08-04,163.44,96.00,67.44,13:49,21:24,5.38,7.58,11,15,71226,71340,114,229.2,3,0,18,Friday,21.562005,78.768
59,2023-08-05,173.75,96.25,77.50,15:42,22:27,5.03,6.75,17,18,71345,71447,102,106.3,3,"(6:28pm, 6:52pm)",18,Saturday,25.740741,30.432
60,2023-08-06,95.00,56.50,38.50,16:37,20:50,3.32,4.22,11,11,71454,71510,56,50.7,3,"(7:04pm, 7:09pm)",18,Sunday,22.511848,20.498
61,2023-08-11,172.25,97.75,74.50,13:54,20:43,5.07,6.80,17,17,71601,71699,98,87.2,3,"(2:55pm, 3:11pm)(5:59pm, 6:07pm)",19,Friday,25.330882,23.158
62,2023-08-12,86.75,51.75,35.00,17:45,21:06,2.55,3.35,9,9,71717,71760,43,33.4,3,0,19,Saturday,25.895522,5.506


In [26]:
sheets_result.head()

,ID,Date,Restaurant Name,Total,Base,Tip,Peak Bonus,Stacked,Version,Day of Week,Start Time,Distance,End Time,Duration,Start Hour,Est Adj
0,2,2023-03-27,Erika's Mexican Food & Seafood,6.75,5.25,1.5,0.0,,1,Monday,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2023-03-27,Chick-fil-A,8.00,4.50,3.5,0.0,,1,Monday,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2023-03-27,Wendy's,7.00,3.00,4.0,0.0,,1,Monday,NaN,NaN,NaN,NaN,NaN,NaN
3,5,2023-03-27,Annam Pho and Sandwiches,7.00,7.00,0.0,0.0,,1,Monday,NaN,NaN,NaN,NaN,NaN,NaN
4,6,2023-03-31,Disfruta,4.75,3.75,1.0,0.0,,1,Friday,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
testingDF = sheets_result.copy()
testingDF1 = testingDF[testingDF["Version"] != 3]
testingDF2 = testingDF[testingDF["Version"] == 3]
def testFunc(df):
    for index, row in df.iterrows():
        if type(row["Stacked"]) == int:
            df.at[index, "Stack Encoding"] = 2
            df.at[index, "Distance"] /= 2
            df.at[index, "Duration"] /= 2
        else:
            if (row["Stacked"] == ""):
                df.at[index, "Stack Encoding"] = 1
            elif (row["Stacked"] == "PENTA"):
                df.at[index, "Stack Encoding"] = 5
                df.at[index, "Distance"] /= 5
                df.at[index, "Duration"] /= 5
            else:
                df.at[index, "Stack Encoding"] = 3
                df.at[index, "Distance"] /= 3
                df.at[index, "Duration"] /= 3
testingDF2["Stack Encoding"] = 0                
testFunc(testingDF2)
testingDF = pd.concat([testingDF1, testingDF2], sort=False)
testingDF.reset_index(drop=True, inplace=True)

ID                                1035
Date               2023-08-04 00:00:00
Restaurant Name               PetSmart
Total                            12.84
Base                               8.3
Tip                               4.54
Peak Bonus                         0.0
Stacked                          PENTA
Version                              3
Day of Week                     Friday
Start Time                  2:49:00 PM
Distance                          37.2
End Time                    4:57:00 PM
Duration                         128.0
Start Hour                         2.0
Est Adj                         42.748
Name: 1033, dtype: object
ID                                1035
Date               2023-08-04 00:00:00
Restaurant Name               PetSmart
Total                            12.84
Base                               8.3
Tip                               4.54
Peak Bonus                         0.0
Stacked                          PENTA
Version                              3

/tmp/ipykernel_5154/3832515498.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testingDF2["Stack Encoding"] = 0


In [29]:
#Group deliveries by date, sum on active mileage  (V3 ONLY)
tempDF = sheets_result[sheets_result["Version"]==3].copy()
tempDF = tempDF.groupby(["Date"]).agg({"Distance": "sum"})
tempDF = tempDF.rename(columns={"Distance": "ActiveMileage"})
tempDF2 = sheets_DaysDF[sheets_DaysDF["Version"]==3].copy()
tempDF3 = tempDF2.merge(tempDF, on=['Date', tempDF2.index])
tempDF3 = tempDF3.drop(columns=['key_1'])
tempDF3
#NOT REJOINED WITH V1 V2

,Date,Total,Base,Tip,Start (24),End (24),Active,Dash,Offers,Deliveries,...,Mileage End,Total Mileage,Active Mileage,Version,Pauses,WeekID,Day of Week,HourlyRate,Est Adj,ActiveMileage
0,2023-05-26,214.69,109.50,105.19,12:20,21:29,6.23,9.15,18,21,...,68110,126,110.1,3,"(3:21pm, 3:45pm)",10,Friday,23.463388,40.074,110.0
1,2023-05-27,180.00,96.00,84.00,13:46,21:18,5.42,7.52,17,17,...,68217,104,88.3,3,"(6:17pm, 6:38pm)",10,Saturday,23.936170,31.582,88.3
2,2023-06-01,135.75,67.25,68.50,16:02,21:50,3.90,5.78,11,14,...,68348,71,70.0,3,0,11,Thursday,23.486159,26.75,70.0
3,2023-06-02,206.53,112.50,94.03,13:02,21:26,6.17,8.40,18,21,...,68472,120,101.8,3,"(5:24pm, 5:53pm)",11,Friday,24.586905,33.172,101.8
4,2023-06-03,208.85,105.00,103.85,14:18,22:34,6.07,8.27,16,18,...,68608,133,102.0,3,"(7:25pm, 7:47pm)",11,Saturday,25.253930,38.94,102.0
5,2023-06-04,101.50,58.00,43.50,16:47,20:50,3.25,4.05,9,10,...,68675,65,59.7,3,0,11,Sunday,25.061728,20.798,59.7
6,2023-06-09,201.29,99.75,101.54,12:20,20:22,5.75,8.37,19,20,...,68865,104,80.6,3,"(1:39pm, 2:09pm)",12,Friday,24.048984,31.154,80.6
7,2023-06-10,181.05,97.50,83.55,16:00,23:22,6.02,7.37,20,21,...,68949,79,81.0,3,0,12,Saturday,24.565807,38.4,81.0
8,2023-06-18,217.05,116.00,101.05,11:40,21:15,6.43,9.57,19,21,...,69192,147,113.6,3,"(12:59pm, 1:17pm)(2:51pm, 3:12pm)",13,Sunday,22.680251,38.364,113.6
9,2023-06-19,241.50,139.50,102.00,12:45,23:36,8.70,10.83,25,27,...,69332,135,136.1,3,"(3:45pm, 4:09pm)",14,Monday,22.299169,63.374,136.1


In [33]:
#Group days by week, sum on active mileage (V3 ONLY)
tempDF4 = tempDF3.copy()
tempDF4 = tempDF4.groupby(["WeekID"]).agg({"ActiveMileage": "sum"})
tempDF5 = sheets_WeeksDF[sheets_WeeksDF["Version"]==3].copy()
tempDF6 = tempDF4.merge(tempDF5, on=['WeekID', tempDF4.index])
tempDF6 = tempDF6.drop(columns=['key_1'])
tempDF6
#NOT REJOINED WITH V1 V2

,WeekID,ActiveMileage,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Total Mileage,Active Mileage,Version,Start Date,End Date,pretotalactiveratio,HourlyRate,Est Adj
0,10,198.3,5/22 - 5/28,205.50,189.19,63.63,0.00,394.69,458.32,11.65,16.67,38,230,198.36,3,5/22,5/28,33.88,27.493701,71.6424
1,11,333.5,5/29 - 6/4,342.75,309.88,115.30,0.00,652.63,767.93,19.40,26.53,63,389,333.5,3,5/29,6/4,33.64,28.945722,119.84
2,12,161.6,6/5 - 6/11,197.25,185.09,65.38,0.00,382.34,447.72,11.77,15.75,41,183,161.6,3,6/5,6/11,32.48,28.426667,69.554
3,13,113.6,6/12 - 6/18,116.00,101.05,34.67,6.50,217.05,258.22,6.43,9.57,21,147,113.6,3,6/12,6/18,33.76,26.982236,38.364
4,14,619.4,6/19 - 6/25,637.25,475.09,240.02,6.25,1112.34,1358.61,38.60,51.23,124,633,619.4,3,6/19,6/25,28.82,26.519813,268.146
5,15,98.1,7/3 - 7/9,114.25,105.10,54.99,7.75,219.35,282.09,7.83,10.17,23,108,98.1,3,7/3,7/9,28.01,27.737463,60.044
6,16,438.6,7/17 - 7/23,454.75,425.11,151.31,3.38,879.86,1034.55,26.22,37.30,91,471,438.6,3,7/17,7/23,33.56,27.735925,166.334
7,17,271.8,7/24 - 7/30,278.25,209.79,71.96,0.00,488.04,560.00,14.82,19.52,50,285,271.8,3,7/24,7/30,32.93,28.688525,80.922
8,18,386.2,7/31 - 8/6,248.75,183.44,74.75,0.00,432.19,506.94,13.75,18.57,44,272,386.2,3,7/31,8/6,31.43,27.298869,130.058
9,19,120.6,8/7 - 8/13,149.50,109.50,0.00,0.00,259.00,259.00,7.62,10.15,26,141,120.6,3,8/7,8/13,33.99,25.517241,28.664
